In [1]:
import numpy
import torch
from stable_baselines3 import SAC
import gym
import mujoco
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import mujoco
import mujoco.viewer
from gym.envs.registration import register
from env import QuadrupedEnv


In [13]:

# if torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")

# # Create the environment
# env = gym.make('Pendulum-v1', render_mode='human')

# # Create the SAC model
# model = SAC('MlpPolicy', env, verbose=1)

# # Train the model
# model.learn(total_timesteps=10000)

# # Save the model
# model.save("sac_pendulum")

# # Load the model (optional)
# model = SAC.load("sac_pendulum")

# # Test the model
# obs, info = env.reset()
# # for _ in range(1000):
# #     action, _states = model.predict(obs)
# #     obs, rewards, dones, truncated, info = env.step(action)
# #     if dones or truncated:
# #         obs, info = env.reset()
# #     env.render()

# Tesbot test

In [14]:
print(gym.envs.registry.keys())

dict_keys(['CartPole-v0', 'CartPole-v1', 'MountainCar-v0', 'MountainCarContinuous-v0', 'Pendulum-v1', 'Acrobot-v1', 'phys2d/CartPole-v0', 'phys2d/CartPole-v1', 'phys2d/Pendulum-v0', 'LunarLander-v3', 'LunarLanderContinuous-v3', 'BipedalWalker-v3', 'BipedalWalkerHardcore-v3', 'CarRacing-v3', 'Blackjack-v1', 'FrozenLake-v1', 'FrozenLake8x8-v1', 'CliffWalking-v0', 'Taxi-v3', 'tabular/Blackjack-v0', 'tabular/CliffWalking-v0', 'Reacher-v2', 'Reacher-v4', 'Reacher-v5', 'Pusher-v2', 'Pusher-v4', 'Pusher-v5', 'InvertedPendulum-v2', 'InvertedPendulum-v4', 'InvertedPendulum-v5', 'InvertedDoublePendulum-v2', 'InvertedDoublePendulum-v4', 'InvertedDoublePendulum-v5', 'HalfCheetah-v2', 'HalfCheetah-v3', 'HalfCheetah-v4', 'HalfCheetah-v5', 'Hopper-v2', 'Hopper-v3', 'Hopper-v4', 'Hopper-v5', 'Swimmer-v2', 'Swimmer-v3', 'Swimmer-v4', 'Swimmer-v5', 'Walker2d-v2', 'Walker2d-v3', 'Walker2d-v4', 'Walker2d-v5', 'Ant-v2', 'Ant-v3', 'Ant-v4', 'Ant-v5', 'Humanoid-v2', 'Humanoid-v3', 'Humanoid-v4', 'Humanoid-v5

In [15]:


class TesbotEnv(gym.Env):
    def __init__(self, render_mode='human'):
        super(TesbotEnv, self).__init__()
        
        # Load your URDF/MJCF model
        self.model = mujoco.MjModel.from_xml_path("tesbot.urdf")
        self.data = mujoco.MjData(self.model)

        # Action and observation spaces (simplified example)
        n_actuators = self.model.nu
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(n_actuators,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(self.model.nq + self.model.nv,), dtype=np.float32)

        # Viewer setup (optional)
        import os
        if render_mode == 'human':
            self.viewer = mujoco.viewer.launch(self.model, self.data)

    def reset(self):
        mujoco.mj_resetData(self.model, self.data)
        return self._get_obs()

    def step(self, action):
        mujoco.mj_set_control(self.model, self.data, action)
        mujoco.mj_step(self.model, self.data)
        obs = self._get_obs()
        reward = self._compute_reward()
        done = False  # Customize based on task
        return obs, reward, done, {}

    def _get_obs(self):
        return np.concatenate([self.data.qpos, self.data.qvel])

    def _compute_reward(self):
        return 1.0  # Placeholder

    def render(self, mode='human'):
        if hasattr(self, 'viewer'):
            self.viewer.sync()

    def close(self):
        if hasattr(self, 'viewer'):
            self.viewer.close()



In [6]:
# Create the environment
env = QuadrupedEnv("quad_ex.xml", render_mode='human')
model = SAC('MlpPolicy', env, verbose=1)

# Train the model
model.learn(total_timesteps=10000)

# Save the model
model.save("sac_tesbot")

# Load the model (optional)
model = SAC.load("sac_tesbot")

# Test the model
obs, info = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs)
    obs, rewards, dones, truncated, info = env.step(action)
    if dones or truncated:
        obs, info = env.reset()
    env.render()


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.097    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 1581     |
|    time_elapsed    | 0        |
|    total_timesteps | 4        |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0965   |
| time/              |          |
|    episodes        | 8        |
|    fps             | 1546     |
|    time_elapsed    | 0        |
|    total_timesteps | 8        |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0971   |
| time/              |          |
|    episodes        | 12       |
|    fps             | 1683     |
|    time_elapsed    | 0        |
|    total_timesteps | 12       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0971   |
| time/              |          |
|    episodes        | 16       |
|    fps             | 1268     |
|    time_elapsed    | 0        |
|    total_timesteps | 16       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0973   |
| time/              |          |
|    episodes        | 20       |
|    fps             | 1318     |
|    time_elapsed    | 0        |
|    total_timesteps | 20       |
--------------

/opt/miniconda3/envs/quad-ex/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0972   |
| time/              |          |
|    episodes        | 112      |
|    fps             | 464      |
|    time_elapsed    | 0        |
|    total_timesteps | 112      |
| train/             |          |
|    actor_loss      | -5.5     |
|    critic_loss     | 0.0012   |
|    ent_coef        | 0.997    |
|    ent_coef_loss   | -0.0403  |
|    learning_rate   | 0.0003   |
|    n_updates       | 11       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | 0.0973   |
| time/              |          |
|    episodes        | 116      |
|    fps             | 406      |
|    time_elapsed    | 0        |
|    total_timesteps | 116      |
| train/             |          |
|    actor_loss      | -5.53    |
|    critic_loss     | 0.000183 |
|    ent_coef 

KeyboardInterrupt: 